# Notebook 4: Mountain Wave Test Case

This notebook will go through an example of nonhydrostatic flow over a mountain. The setup is described in the paper avaliable at this link: https://doi.org/10.1002/qj.603. 

The new features in this example are related to the introduction of the mountain. This requires that we deform the mesh so that the bottom of the domain follows the specified mountain profile. We then have the issue that when we calculate the initial balanced conditions, we no longer have a flat bottom on which to specify the required bottom boundary condition. We solve this problem by applying the boundary condition at the top and then applying a shift so that the bottom boundary condition is as required. The final new feature is the sponge layer which is required at the top of the domain to damp vertically propagating waves so that they are not reflected back into the domain by the nonphysical impermeable upper boundary.

We begin by importing the required functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, VectorFunctionSpace,
                       PeriodicIntervalMesh, ExtrudedMesh, SpatialCoordinate,
                       exp, pi, cos, Function, conditional, Mesh)

INFO     Running /Users/JS1075/firedrake_dec24/lib/python3.12/site-packages/ipykernel_launcher.py -f /private/var/folders/f0/llvlmlb50qg6mmlxs8m6d6d00000gp/T/tmposdw7vut.json --HistoryManager.hist_file=:memory:


Define the time-step size and the duration of the simulation. 

In [2]:
dt = 5.0
tmax = 9000.

Like with the gravity wave example, we will create a periodic interval mesh in the horizontal and extrude this in the vertical dimension to create a two dimensional mesh.

In [3]:
nlayers = 70  # horizontal layers
columns = 180  # number of columns
domain_width = 144000.
base_mesh = PeriodicIntervalMesh(columns, domain_width)

domain_height = 35000.  # Height position of the model top
ext_mesh = ExtrudedMesh(base_mesh, layers=nlayers,
                        layer_height=domain_height/nlayers)

Vc = VectorFunctionSpace(ext_mesh, "DG", 2)
coord = SpatialCoordinate(ext_mesh)
x = Function(Vc).interpolate(as_vector([coord[0], coord[1]]))
a = 1000.
xc = L/2.
x, z = SpatialCoordinate(ext_mesh)
hm = 1.
zs = hm*a**2/((x-xc)**2 + a**2)

NameError: name 'L' is not defined

Now we transform the mesh to follow the defined terrain. A smoothing is applied in the vertical direction.

In [4]:
dirname = 'nh_mountain'

zh = 5000.
xexpr = as_vector([x, conditional(z < zh, z + cos(0.5*pi*z/zh)**6*zs, z)])

new_coords = Function(Vc).interpolate(xexpr)
mesh = Mesh(new_coords)
mesh._base_mesh = base_mesh  # Force new mesh to inherit original base mesh
domain = Domain(mesh, dt, family="CG", degree=1)

NameError: name 'z' is not defined

In [5]:
g = 9.80665              # acceleration due to gravity, in m/s^2
cp = 1004.               # specific heat capacity at constant pressure
parameters = CompressibleParameters(g=g, cp=cp)
sponge_depth = 10000.0   # depth of sponge layer, in m
sponge_mu = 0.15         # parameter for strength of sponge layer, in J/kg/K
sponge = SpongeLayerParameters(H=domain_height, z_level=domain_height-sponge_depth, mubar=sponge_mu/dt)
eqns = CompressibleEulerEquations(domain, parameters, sponge_options=sponge)

AttributeError: module 'gusto.core.domain' has no attribute 'spaces'

We will record the full velocity fields, as well as the perturbations to the potential temperature and density.

In [6]:
output = OutputParameters(dirname=dirname, dumpfreq=10, dump_vtus=True, dump_nc=True)
diagnostic_fields = [Exner(parameters), ZComponent('u'), Perturbation('theta'), Perturbation('rho')]
io = IO(domain, output, diagnostic_fields=diagnostic_fields)

AttributeError: module 'gusto.core.domain' has no attribute 'mesh'

Set up the corresponding state:

In [7]:
theta_opts = SUPGOptions()
transported_fields = [TrapeziumRule(domain, "u"), SSPRK3(domain, "rho"), SSPRK3(domain, "theta", options=theta_opts)]
transport_methods = [DGUpwind(eqns, "u"), DGUpwind(eqns, "rho"), DGUpwind(eqns, "theta", ibp=theta_opts.ibp)]

linear_solver = CompressibleSolver(eqns)
stepper = SemiImplicitQuasiNewton(eqns, io,
                                  transported_fields,
                                  transport_methods,
                                  linear_solver=linear_solver)

AttributeError: module 'gusto.core.domain' has no attribute 'dt'

In [8]:
# Initial conditions
u0 = state.fields("u")
rho0 = state.fields("rho")
theta0 = state.fields("theta")

# spaces
Vu = state.spaces("HDiv")
Vt = state.spaces("theta")
Vr = state.spaces("DG")

NameError: name 'state' is not defined

Set up the relevant parameters for the test case. Most of these relate to thermodynamic conditions.

In [9]:
# Thermodynamic constants required for setting initial conditions
# and reference profiles
g = parameters.g
N = parameters.N
p_0 = parameters.p_0
c_p = parameters.cp
R_d = parameters.R_d
kappa = parameters.kappa

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
xz = SpatialCoordinate(mesh)
Tsurf = 300.
thetab = Tsurf*exp(N**2*xz[1]/g)
theta_b = Function(Vt).interpolate(thetab)

NameError: name 'mesh' is not defined

Determine the Exner pressure using the hydrostatic balance function. (Is this because we start with a hydrostatic balance, then allow the simulation to move away from this?)

In [10]:
# Thermodynamic constants required for setting initial conditions
# and reference profiles
N = parameters.N

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
xz = SpatialCoordinate(mesh)
thetab = Tsurf*exp(N**2*z/g)
theta_b = Function(Vt).interpolate(thetab)

# Calculate hydrostatic exner
exner = Function(Vr)
rho_b = Function(Vr)

# Set up kernels to evaluate global minima and maxima of fields
min_kernel = MinKernel()
max_kernel = MaxKernel()

# First solve hydrostatic balance that gives Exner = 1 at bottom boundary
# This gives us a guess for the top boundary condition
bottom_boundary = Constant(exner_surf, domain=mesh)
logger.info(f'Solving hydrostatic with bottom Exner of {exner_surf}')
compressible_hydrostatic_balance(
    eqns, theta_b, rho_b, exner, top=False, exner_boundary=bottom_boundary
)

# Solve hydrostatic balance again, but now use minimum value from first
# solve as the *top* boundary condition for Exner
top_value = min_kernel.apply(exner)
top_boundary = Constant(top_value, domain=mesh)
logger.info(f'Solving hydrostatic with top Exner of {top_value}')
compressible_hydrostatic_balance(
    eqns, theta_b, rho_b, exner, top=True, exner_boundary=top_boundary
)

max_bottom_value = max_kernel.apply(exner)

# Now we iterate, adjusting the top boundary condition, until this gives
# a maximum value of 1.0 at the surface
lower_top_guess = 0.9*top_value
upper_top_guess = 1.2*top_value
for i in range(max_iterations):
    # If max bottom Exner value is equal to desired value, stop iteration
    if abs(max_bottom_value - exner_surf) < tolerance:
        break

    # Make new guess by average of previous guesses
    top_guess = 0.5*(lower_top_guess + upper_top_guess)
    top_boundary.assign(top_guess)

    compressible_hydrostatic_balance(
        eqns, theta_b, rho_b, exner, top=True, exner_boundary=top_boundary
    )

    max_bottom_value = max_kernel.apply(exner)

    # Adjust guesses based on new value
    if max_bottom_value < exner_surf:
        lower_top_guess = top_guess
    else:
        upper_top_guess = top_guess

# Perform a final solve to obtain hydrostatically balanced rho
compressible_hydrostatic_balance(
    eqns, theta_b, rho_b, exner, top=True, exner_boundary=top_boundary,
    solve_for_rho=True
)

theta0.assign(theta_b)
rho0.assign(rho_b)
u0.project(as_vector([initial_wind, 0.0]), bcs=eqns.bcs['u'])

stepper.set_reference_profiles([('rho', rho_b), ('theta', theta_b)])

stepper.run(t=0, tmax=tmax)

NameError: name 'mesh' is not defined

Now we would run the simulation! Note, that it does take a while ... . 

In [11]:
# Here's what you would do:
# stepper.run(t=0, tmax=tmax)